<a href="https://colab.research.google.com/github/aflip/IDSP/blob/main/IDSP_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


This research is funded by Medbuddy-Docsapp

## [DocsApp -- Talk to a Specialist Doctor 24x7 - Download now](https://play.google.com/store/apps/details?id=com.docsapp.patients&hl=en_IN)


The library I settled on after experimenting with most of the well maintained one is Camelot.py by Vinayak Mehta. It features a very intutive API, really good visual debugging tools extensive docs which make life a lot easier.  It's brilliant and you can find out more at https://camelot-py.readthedocs.io

It so happens that Vinayak has a notebook in which he applies camelot to extracting IDSP data, and so a lot of the code I have used in this notebook is his. Source notebook is here: https://gist.github.com/vinayak-mehta/e5949f7c2410a0e12f25d3682dc9e873 

I used the firefox addon DownloadThemAll https://addons.mozilla.org/en-US/firefox/addon/downthemall/ to extract the data from IDSP website.  I would highlight a particular year and save that to a folder.  

I had also tried extracting using the source notebook downloader, but kept getting timed out. Also tried HTTRACK but DTA was the best.



## Pre Code workflow
1. Download PDF files from https://idsp.nic.in/index4.php?lang=1&level=0&linkid=406&lid=3689 into yearly folders using DTA
2. Split each year's folder into 2 folders of around 25 files each (to aboid ram running out)
3. Copy one or two files from each year into a test folder to run the experiments from, after which one can run the code on the whole year
4. Set colab to a high ram instance. I have a Colab subscription, which allows me to do this. YMMV. It takes roughly as much Ram in GBs as the number of files being processed in one run, so you might want to split up the folders smaller/larger depending. 


# Set up the system

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!apt install python-tk ghostscript # GS is a prerequisite for Camelot

In [ ]:
!pip install camelot-py # This is the library used to do the extraction

In [ ]:
! pip freeze --local > /content/gdrive/My\ Drive/colab_installed.txt 
# So that I have a handy list to reinstall the env from

In [ ]:
# run this for reruns only add --force-reinstall to upgrade if it's been a long time
! pip install --upgrade `cat /content/gdrive/My\ Drive/colab_installed.txt`

Requirement already up-to-date: absl-py==0.9.0 in /usr/local/lib/python3.6/dist-packages (0.9.0)
Requirement already up-to-date: alabaster==0.7.12 in /usr/local/lib/python3.6/dist-packages (0.7.12)
Requirement already up-to-date: albumentations==0.1.12 in /usr/local/lib/python3.6/dist-packages (0.1.12)
Requirement already up-to-date: altair==4.1.0 in /usr/local/lib/python3.6/dist-packages (4.1.0)
Requirement already up-to-date: argon2-cffi==20.1.0 in /usr/local/lib/python3.6/dist-packages (20.1.0)
Requirement already up-to-date: asgiref==3.2.10 in /usr/local/lib/python3.6/dist-packages (3.2.10)
Requirement already up-to-date: astor==0.8.1 in /usr/local/lib/python3.6/dist-packages (0.8.1)
Requirement already up-to-date: astropy==4.0.1.post1 in /usr/local/lib/python3.6/dist-packages (4.0.1.post1)
Requirement already up-to-date: astunparse==1.6.3 in /usr/local/lib/python3.6/dist-packages (1.6.3)
Requirement already up-to-date: atari-py==0.2.6 in /usr/local/lib/python3.6/dist-packages (0.2

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import re
import glob

In [ ]:
import camelot as cm

# Experiment


This part is to experiment with the API and figure out how well the different options are giving you results. I got the best results from using Stream with a Row tolerance between 55 and 70. The following cells are some ways in which one can visually explore the extraction. More details can be found at https://camelot-py.readthedocs.io/en/master/user/advanced.html 

In [ ]:
tables= cm.read_pdf('data/2-end/5.pdf', pages='11-13' , flavor = 'lattice', line_scale=70)

In [ ]:
tables= cm.read_pdf('data/2-end/5.pdf', pages='11-13' , flavor = 'stream', row_tol=50)

In [ ]:
cm.plot(tables[0], kind='grid')

In [ ]:
cm.plot(tables[0], kind='contour')

In [ ]:
 tables.export('1s.csv', f='csv', compress=False)  
 # inspecting the csvs was the fastest way to get detailed info

# Factory Method

## Set up extraction


In [ ]:
# The lables for the columns in the various files are slightly different or are in different orders, 
# this code helps identify each and standardize. 
# this is a heuristic copied from Vinayak Mehta
headers = [
    'unique_id',
    'state',
    'district',
    'disease_illness',
    'num_cases',
    'num_deaths',
    'date_of_start_of_outbreak',
    'date_of_reporting',
    'current_status',
    'comment_action_taken',
    'reported_late',
    'under_surveillance'
]
ten_headers = [
    'unique_id',
    'state',
    'district',
    'disease_illness',
    'num_cases',
    'num_deaths',
    'date_of_start_of_outbreak',
    'date_of_reporting',
    'current_status',
    'comment_action_taken'
]
nine_headers = [
    'unique_id',
    'state',
    'district',
    'disease_illness',
    'num_cases',
    'num_deaths',
    'date_of_start_of_outbreak',
    'current_status',
    'comment_action_taken'
]

In [ ]:
# Takes the tables we extract and makes clean DF from from it. 

def append_tables(table_list):
    df = pd.DataFrame(columns=headers)
    for tables in all_tables:
        for table in tables:
            columns = list(table.df.iloc[0])
            if table.shape[1] == 10:
                temp = table.df.copy()
                if 'unique' in columns[0].lower():
                    temp = temp.iloc[1:]
                temp.columns = ten_headers
                temp['reported_late'] = False
                temp['under_surveillance'] = False
                df = pd.concat([df, temp], sort=False)
            elif table.shape[1] == 9:
                temp = table.df.copy()
                if 'disease' in columns[0].lower():
                    c = temp.iloc[0]
                    temp = temp.iloc[2:]
                    temp.columns = nine_headers
                    if 'reportedlate' in c[0].lower().replace(' ', ''):    
                        temp['reported_late'] = True
                        temp['under_surveillance'] = False
                    elif 'undersurv' in c[0].lower().replace(' ', ''):
                        temp['reported_late'] = False
                        temp['under_surveillance'] = True
                    df = pd.concat([df, temp], sort=False)
                else:
                    temp.columns = nine_headers
                    temp['reported_late'] = True
                    temp['under_surveillance'] = False
                    df = pd.concat([df, temp], sort=False)
    return df


## Run the code

After experimentation it was obvious that even with a high-ram instance in colab (25GB), or on my system with 16GB RAM, processing more than 25 files in one go would crash the system due to inavailability of ram.  So I split up each year into two folders of 25-26 files each and appended the tables into one before processing. With the settings so far, and 25 gb ram, each cyle took between 5 and 15 minutes. so the whole 12 years of data took around 6 hours. Most of the time, of course was spent figuring out how to get it to this point.


In [ ]:
all_tables = []
def extract_tables_from_pdfs():
    for file in glob.iglob(r'/content/gdrive/My Drive/Colab Notebooks/IDSP_Data/data/2020/1/*.pdf'):
        tables = cm.read_pdf(file, pages="3-end", flavor = 'lattice',  line_scale=40)
        print( 'Found {} tables(s)'.format(tables.n))
        all_tables.append(tables)
    return all_tables

# for recursively looking through a folder including subfolders use pattern
# `for file in glob.iglob(r'/data_folder/**/*.pdf')`


In [ ]:
# I like timing things
%time extract_tables_from_pdfs()

In [ ]:
df_2020_1 = append_tables(all_tables)

In [ ]:
df_2020_2 = append_tables(all_tables)

In [ ]:
df_2020 = df_2020_1.append(df_2020_2)

## Clean & export data

This does only the most basic clean up, further clearning depends on the type of analysis you want to do, and for me, that was much more extensive and depended on the year of analysis so I am not including that here. This is how I got to the shared files.


In [ ]:
df_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 514 entries, 1 to 5
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   unique_id                  514 non-null    object
 1   state                      514 non-null    object
 2   district                   514 non-null    object
 3   disease_illness            514 non-null    object
 4   num_cases                  514 non-null    object
 5   num_deaths                 514 non-null    object
 6   date_of_start_of_outbreak  514 non-null    object
 7   date_of_reporting          301 non-null    object
 8   current_status             514 non-null    object
 9   comment_action_taken       514 non-null    object
 10  reported_late              499 non-null    object
 11  under_surveillance         499 non-null    object
dtypes: object(12)
memory usage: 52.2+ KB


In [ ]:
# Came to just these after much trial and error
def dfcleaning(df):
  df = df.drop(['reported_late' , 'under_surveillance'], axis=1) # These are generated by the extraction function for keeping track of things and are not of use
  df['date_of_reporting'] = df['date_of_reporting'].fillna("n/a") # Fill with whatever you'd like, this helps clean it up in a spreadsheet software easier 
  df = df.drop(df[df.unique_id == "U\nnique I.D."].index)  #This takes care of the repeated header rows
  df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x) # Removes trailing and inital whitespaces
  return df


In [ ]:
df_2020 = df_2020.pipe(dfcleaning) # discovering pandas' pipe function was one of the highlights of this process. 

In [ ]:
df_2020[1400:1420]

In [ ]:
df_2020.to_csv('/content/gdrive/My Drive/Colab Notebooks/IDSP_Data/df_2020.csv',  index= False)

In [ ]:
del  df_2020